# Sora2 Watermark Remover

AI を使って Sora 動画からウォーターマークを除去します。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover


In [10]:
# リポジトリのクローンとブランチ切替
!git clone https://github.com/fulfulggg/Sora2WatermarkRemover.git
%cd Sora2WatermarkRemover
!git fetch origin --prune
!git checkout main
!git pull --ff-only
print("✅ リポジトリの準備完了")


Cloning into 'Sora2WatermarkRemover'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 194 (delta 89), reused 180 (delta 77), pack-reused 0 (from 0)
Receiving objects: 100% (194/194), 104.91 KiB | 14.99 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/Sora2WatermarkRemover/Sora2WatermarkRemover
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
✅ リポジトリの準備完了


## ⚠️ 重要: 依存関係の修正

次のセルを実行後、**必ずランタイムを再起動**してください。


In [11]:
# Pillow 修正（実行後にランタイムを再起動）
%pip uninstall -y pillow PIL
%pip install -U "pillow==10.4.0"
print("\n✅ 完了。ランタイム→ランタイムを再起動 を実行してください。")


Found existing installation: Pillow 9.5.0
Uninstalling Pillow-9.5.0:
  Successfully uninstalled Pillow-9.5.0
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
iopaint 1.2.2 requires Pillow==9.5.0, but you have pillow 10.4.0 which is incompatible.



✅ 完了。ランタイム→ランタイムを再起動 を実行してください。


In [1]:
# 再起動後の確認
import PIL
import PIL._util as u

print(f"Pillow: {PIL.__version__}")
assert PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"), "❌ ランタイムを再起動してください"
print("✅ 依存関係OK")


Pillow: 10.4.0
✅ 依存関係OK


In [2]:
# 競合除去
%pip uninstall -y diffusers huggingface_hub transformers

# ベースライブラリ（ColabのGPU PyTorchを維持するためtorchは入れない）
%pip install -q "iopaint==1.2.2" opencv-python tqdm loguru

# 安定版ピン（Florence trust_remote_codeとiopaintの両立）
%pip install -q "diffusers==0.26.3" "huggingface_hub==0.25.2" "transformers==4.42.4"

!apt-get -qq install -y ffmpeg

# バージョン確認
from importlib.metadata import version as v
print(f"transformers: {v('transformers')}")
print(f"huggingface_hub: {v('huggingface_hub')}")
print(f"diffusers: {v('diffusers')}")
print(f"iopaint: {v('iopaint')}")
print("✅ パッケージインストール完了")


Found existing installation: diffusers 0.26.3
Uninstalling diffusers-0.26.3:
  Successfully uninstalled diffusers-0.26.3
Found existing installation: huggingface-hub 0.25.2
Uninstalling huggingface-hub-0.25.2:
  Successfully uninstalled huggingface-hub-0.25.2
Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
google-adk 1.15.1 requires fastapi<1.0.0,>=0.115.0, but you have fastapi 0.108.0 which is incompatible.
google-adk 1.15.1 requires starlette<1.0.0,>=0.46.2, but you have starlette 0.32.0.post1 which is incompatible.
google-adk 1.15.1 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
yfinance 0.2.66

In [3]:
# LaMa モデルの事前ダウンロード
!iopaint download --model lama
print("✅ LaMaモデルのダウンロード完了")


2025-10-10 00:16:40.170 | INFO     | iopaint.runtime:setup_model_dir:82 - Model directory: /root/.cache
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is depr

In [4]:
%%bash
set -e
cd /content/Sora2WatermarkRemover
mkdir -p flash_attn
cat > flash_attn/__init__.py <<'PY'
"""
Lightweight stub package for `flash_attn`.
Satisfies import-time checks in Florence-2 (trust_remote_code). Not a real implementation.
"""
def __getattr__(name: str):
    raise RuntimeError(
        f"flash_attn stub in use: attempted to access attribute '{name}'. "
        "This environment runs with eager attention. If you need FlashAttention, "
        "install a matching wheel for your CUDA/SM."
    )
PY
echo "✅ flash_attn stub created at /content/Sora2WatermarkRemover/flash_attn"


✅ flash_attn stub created at /content/Sora2WatermarkRemover/flash_attn


In [5]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU未検出。ランタイム→ランタイムのタイプを変更→GPU")


Device: cuda
GPU: NVIDIA A100-SXM4-40GB


In [6]:
from google.colab import files

print("動画ファイルをアップロード:")
uploaded = files.upload()
input_video = list(uploaded.keys())[0]
print(f"✅ アップロード: {input_video}")


動画ファイルをアップロード:


Saving download.MP4 to download (1).MP4
✅ アップロード: download (1).MP4


## 処理セル（通常）
パラメータ設定
- `max_bbox_percent`: 検出する最大サイズ（画像比%）
- `frame_step`: 処理間隔（1=全フレーム、2=隔フレーム）
- `target_fps`: 出力fps（0=入力と同じ）


In [9]:
# パラメータ
max_bbox_percent = 22.0
frame_step = 1
target_fps = 0.0
output_video = "output.mp4"

# 処理実行
import os
import time

input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

%cd /content/Sora2WatermarkRemover

print(f"入力: {input_path}")
print(f"出力: {output_path}")
print(f"パラメータ: bbox={max_bbox_percent}%, step={frame_step}, fps={target_fps}")
print("\n🚀 処理開始...\n")

t0 = time.time()
!python remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 7 \
  --mask-dilate 6

elapsed = time.time() - t0

if os.path.exists(output_path):
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n✅ 処理完了")
    print(f"⏱ 処理時間: {elapsed:.1f}秒")
    print(f"📊 ファイルサイズ: {size_mb:.2f} MB")
else:
    print("\n❌ エラー: 出力ファイルが作成されませんでした")


/content/Sora2WatermarkRemover
入力: /content/download (1).MP4
出力: /content/output.mp4
パラメータ: bbox=24.0%, step=1, fps=0.0

🚀 処理開始...

2025-10-10 00:34:59.460154: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-10 00:34:59.477713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760056499.498839   30381 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760056499.505327   30381 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00

In [ ]:
# パターンB：強め（まだ残る場合）
# bbox=24, temporal=7, dilate=6
# 検出範囲を広げて取りこぼしを減らす

max_bbox_percent = 24.0
frame_step = 1
target_fps = 0.0
output_video = "output_strong.mp4"

import os
import time

input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

%cd /content/Sora2WatermarkRemover

print(f"入力: {input_path}")
print(f"出力: {output_path}")
print(f"パターン: B（強め） bbox={max_bbox_percent}%, step={frame_step}, fps={target_fps}")
print("\n🚀 処理開始...\n")

t0 = time.time()
!python remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 7 \
  --mask-dilate 6

elapsed = time.time() - t0

if os.path.exists(output_path):
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n✅ 処理完了")
    print(f"⏱ 処理時間: {elapsed:.1f}秒")
    print(f"📊 ファイルサイズ: {size_mb:.2f} MB")
else:
    print("\n❌ エラー: 出力ファイルが作成されませんでした")


## 処理セル（２パス方式）

In [10]:
# パラメータ
max_bbox_percent = 24.0
frame_step = 1
target_fps = 0.0
output_video = "output.mp4"

# 処理実行（2パス方式）
import os
import time

input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

%cd /content/Sora2WatermarkRemover

print(f"入力: {input_path}")
print(f"出力: {output_path}")
print(f"パラメータ: bbox={max_bbox_percent}%, step={frame_step}, fps={target_fps}")
print("\n🚀 2パス処理開始...\n")

t0 = time.time()

# パス1（中程度で大枠除去）
print("--- パス1：中程度で大枠除去 ---")
!python remwm.py "{input_path}" "/content/pass1.mp4" \
  --max-bbox-percent 20 \
  --frame-step 1 \
  --target-fps 0 \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 5 \
  --mask-dilate 5

# パス2（強めで残渣を拾う）
print("\n--- パス2：強めで残渣を拾う ---")
!python remwm.py "/content/pass1.mp4" "{output_path}" \
  --max-bbox-percent 26 \
  --frame-step 1 \
  --target-fps 0 \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 9 \
  --mask-dilate 7

elapsed = time.time() - t0

# 結果確認
if os.path.exists(output_path):
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n✅ 2パス完了")
    print(f"⏱ 処理時間: {elapsed:.1f}秒")
    print(f"📊 最終サイズ: {size_mb:.2f} MB")
else:
    print("\n❌ エラー: 出力ファイルが作成されませんでした")

/content/Sora2WatermarkRemover
入力: /content/download (1).MP4
出力: /content/output.mp4
パラメータ: bbox=24.0%, step=1, fps=0.0

🚀 2パス処理開始...

--- パス1：中程度で大枠除去 ---
2025-10-10 00:42:08.023723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-10 00:42:08.041341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760056928.062427   32305 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760056928.068736   32305 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already bee

In [11]:
# 結果のダウンロード
from google.colab import files

files.download(f"/content/{output_video}")
print("✅ ダウンロード完了")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ ダウンロード完了


## 使い方のヒント

### 処理を高速化
```python
frame_step = 2  # 隔フレーム処理
```

### 出力fpsを固定
```python
target_fps = 30
```

### 検出感度を調整
```python
max_bbox_percent = 15  # 大きいロゴも検出
```

---

## チューニングガイド（点滅対策）

**時間安定化（temporal-mask/mask-dilate）は既定ONで実装済み**です。以下の順で試してください。

### ステップ1：既定値で実行（まずこれ）

```python
# パラメータ
max_bbox_percent = 18.0  # 検出上限
frame_step = 1           # 全フレーム処理
target_fps = 0.0         # 入力と同じfps

# 既存の実行セルをそのまま使用
# （既定で temporal=3, dilate=4 が有効）
```

**確認**: 点滅が大幅に減っているか → まだ残る場合はステップ2へ

---

### ステップ2：強め設定（点滅が残る場合）

```python
# パラメータ（変更）
max_bbox_percent = 20.0  # 検出範囲を広げる

# 実行（temporal-mask/mask-dilateを明示）
!python remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 5 \
  --mask-dilate 5
```

**効果**:
- `temporal-mask 5`: 直近5フレームのマスクを合成（検出揺らぎに強い）
- `mask-dilate 5`: マスクを5px膨張（縁の取りこぼし防止）

---

### ステップ3：さらに強め（頑固なケース）

```python
# パラメータ（さらに調整）
max_bbox_percent = 22.0

# 実行（時間窓を最大に）
!python remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 7 \
  --mask-dilate 6
```

---

### ステップ4：2パス方式（最も確実）

1フレームだけ残る取りこぼしを、2回の処理で確実に拾う：

```python
# パス1（中程度で大枠除去）
!python remwm.py "{input_path}" "/content/pass1.mp4" \
  --max-bbox-percent 18 \
  --frame-step 1 \
  --target-fps 0 \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 5 \
  --mask-dilate 5

# パス2（強めで残渣を拾う）
!python remwm.py "/content/pass1.mp4" "{output_path}" \
  --max-bbox-percent 24 \
  --frame-step 1 \
  --target-fps 0 \
  --force-format=MP4 \
  --overwrite \
  --temporal-mask 9 \
  --mask-dilate 6

# 結果確認
if os.path.exists(output_path):
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"✅ 2パス完了")
    print(f"📊 最終サイズ: {size_mb:.2f} MB")
```

---

### 検証のコツ

**1. 短尺クリップで試す**（時間短縮）
```python
# 冒頭10秒だけ切り出し
!ffmpeg -i "{input_path}" -t 10 -c copy /content/test_clip.mp4
input_video = "test_clip.mp4"
```

**2. パラメータの意味**
- `max_bbox_percent`: 大きくすると広い範囲を検出（偽陽性リスクも上がる）
- `temporal-mask`: 大きくすると時間方向の安定化が強まる
- `mask-dilate`: 大きくすると縁が厚くなる（除去範囲が若干広がる）

**3. 推奨フロー**
1. 既定で実行 → 目視確認
2. まだ点滅あり → ステップ2（強め）
3. まだ残る → ステップ3（さらに強め）
4. 頑固 → ステップ4（2パス）

---

### 最終手段（まだ残る場合）

**RIFE で事前にfps補間 → 全フレーム処理**
- 重いが最も滑らか・取りこぼしが少ない
- fps×2/×4 補間後に本ツールで処理
